In [61]:
import pandas as pd
import numpy as np

title_akas = pd.read_csv('../downloads/title.akas.tsv', delimiter='\t', dtype={'isOriginalTitle': object})
title_basics = pd.read_csv('../downloads/title.basics.tsv', delimiter='\t', dtype={'isAdult': object})
title_crew = pd.read_csv('../downloads/title.crew.tsv', delimiter='\t', dtype={'directors': str})
title_ratings = pd.read_csv('../downloads/title.ratings.tsv', delimiter='\t')
name_basics = pd.read_csv('../downloads/name.basics.tsv', delimiter='\t', dtype={'primaryName': str})

In [62]:
# filter by titleType

title_basics = title_basics[(title_basics['titleType'] == 'movie') | (title_basics['titleType'] == 'tvMovie')]
title_basics.reset_index(inplace=True)

# filter by isOriginalTitle

title_akas = title_akas[(title_akas['isOriginalTitle'] == '1') | (title_akas['isOriginalTitle'] == '\\N')]
title_akas.reset_index(inplace=True)


In [63]:
# remove duplicate values with null value in 'isOriginalTitle' column

duplicates = title_akas[title_akas.duplicated(['titleId'])]
duplicates = duplicates[(duplicates['isOriginalTitle'] == '\\N') | (duplicates['attributes'] != '\\N')]

title_akas.drop(labels=duplicates.index, axis=0, inplace=True)
title_akas.reset_index(inplace=True)

In [64]:
duplicates = title_akas[title_akas.duplicated(['titleId'])]

title_akas.drop(labels=duplicates.index, axis=0, inplace=True)
title_akas.reset_index(inplace=True, drop=True)

In [65]:
# drop useless columns 

title_akas.drop(labels=['ordering', 'types', 'attributes', 'title', 'isOriginalTitle'], axis=1, inplace=True)
title_basics.drop(labels=['endYear', 'titleType'], axis=1, inplace=True)
title_crew.drop(labels=['writers'], axis=1, inplace=True)
title_ratings.drop(labels=['numVotes'], axis=1, inplace=True)
name_basics.drop(labels=['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], axis=1, inplace=True)

In [66]:
title_akas['language'].value_counts()

\N     1790606
en           4
tr           1
yue          1
da           1
ca           1
Name: language, dtype: int64

:warning:  language column values have little to none importance, hence not using this dataframe

In [67]:
# If it is to remove movies before a certain year
# Remove from title_basics


#REVIEW
# I don't think we should remove them 
# Bia

In [68]:
df = title_basics.merge(
  title_crew, on='tconst', how='left').merge(
    title_ratings, on='tconst', how='left')

In [69]:
# explode movie rows with more than one director into multiple rows, each one with only one director code

df = df.assign(directors=df['directors'].str.split(',')).explode('directors')
df = df.merge(name_basics, left_on='directors', right_on='nconst', how='left')

# drop useless columns

df.drop(labels=['directors', 'nconst'], axis=1, inplace=True)
df.rename(columns={'primaryName' : 'directors'}, inplace=True)

In [72]:
# remove duplicate rows while joining director names
df['directors'] = df['directors'].astype(str)
directors_column = df.groupby(['tconst']).agg({'directors': ', '.join})['directors'].values

df.drop_duplicates(subset='tconst', inplace=True)
df['directors'] = directors_column
df.reset_index(inplace=True)

df

# TODO:
- remover filmes mais antigos que x data
- filtrar filmes apenas em en 
  - not enough values
- remover filmes sem sinopse (isto é quando formos fazer o webscrapping ig)
